In [1]:
import requests
from azure.identity import DefaultAzureCredential

# Replace with your actual subscription ID and location
subscription_id = "4d89db8e-5a7e-4462-af2b-fc9bc225f338"
location = "CentralUS"
api_version = "2024-10-01"

# Construct the endpoint URL
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.CognitiveServices/locations/{location}/models?api-version={api_version}"

# Authenticate using Azure Identity (requires Azure CLI login or environment credentials)
credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check and print results
if response.status_code == 200:
    models = response.json()
    print(models)
    for model in models.get("value", []):
        print(f"Model Name: {model['model']['name']}, Version: {model['model']['version']}, Lifecycle: {model['model']['lifecycleStatus']}, Deprecation Date: {model['model']['deprecation'].get('inference', 'N/A')}   ")
else:
    print(f"Error {response.status_code}: {response.text}")

{'value': [{'id': '/subscriptions/4d89db8e-5a7e-4462-af2b-fc9bc225f338/providers/Microsoft.CognitiveServices/locations/CentralUS/models/AI21 Labs.AI21-Jamba-1.5-Large.1', 'type': 'Microsoft.CognitiveServices/locations/models', 'name': 'AI21 Labs.AI21-Jamba-1.5-Large.1', 'location': 'CentralUS', 'kind': 'MaaS', 'skuName': 'S0', 'model': {'format': 'AI21 Labs', 'name': 'AI21-Jamba-1.5-Large', 'version': '1', 'isDefaultVersion': False, 'skus': [{'name': 'GlobalStandard', 'usageName': 'AIServices.GlobalStandard.MaaS', 'capacity': {'maximum': 1, 'default': 1}}], 'maxCapacity': 3, 'capabilities': {'chatCompletion': 'true'}, 'deprecation': {'inference': '2099-12-31T00:00:00Z'}, 'lifecycleStatus': 'Stable', 'systemData': {'createdBy': 'Microsoft', 'createdByType': 'Application', 'createdAt': '2024-10-01T00:00:00Z', 'lastModifiedBy': 'MaaSModelConverter', 'lastModifiedByType': 'Application', 'lastModifiedAt': '2025-04-16T04:45:33.9367873Z'}}}, {'id': '/subscriptions/4d89db8e-5a7e-4462-af2b-fc9b

In [3]:
import requests
import pyodbc
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

load_dotenv()

# === SQL Server Connection ===
server = 'cnt-demo-db-2.database.windows.net'
database = 'demodb'
username = os.getenv("SQL_USERNAME")
password = os.getenv("SQL_PASSWORD")
driver = 'ODBC Driver 17 for SQL Server'

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

# === Azure API Call ===
subscription_id = os.getenv("SUBSCRIPTION_ID")
location = "EastUS"
api_version = "2024-10-01"
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.CognitiveServices/locations/{location}/models?api-version={api_version}"

credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

cursor = conn.cursor()
# === Upsert Each Model ===
if response.status_code == 200:
    models = response.json().get("value", [])
    for model in models:
        model_name = model['model']['name']
        model_version = model['model']['version']
        lifecycle_status = model['model']['lifecycleStatus']
        deprecation_date = model['model']['deprecation'].get('inference', 'N/A')
        name = model['name']
        location = model['location']
        model_format = model['model']['format']

        cursor.execute(f"""
                MERGE models AS target
                USING (SELECT '{model_name}' AS model_name, '{model_version}' AS model_version, '{lifecycle_status}' AS lifecycle_status, 
                    '{deprecation_date}' AS deprecation_date, '{name}' AS name, '{location}' AS location, '{model_format}' AS model_format) AS source
                ON target.model_name = source.model_name 
                AND target.model_version = source.model_version 
                AND target.lifecycle_status = source.lifecycle_status 
                AND target.deprecation_date = source.deprecation_date
                AND target.location = source.location
                WHEN MATCHED THEN
                    UPDATE SET last_update_date = GETDATE()
                WHEN NOT MATCHED THEN
                    INSERT (name, location, model_name, model_version, lifecycle_status, deprecation_date, model_format)
                    VALUES (source.name, source.location, source.model_name, source.model_version, source.lifecycle_status, source.deprecation_date, source.model_format);
           """)

        conn.commit()
else:
    print(f"Error {response.status_code}: {response.text}")

cursor.close()
conn.close()